In [4]:
USE ALM_TEST;
GO

CREATE OR ALTER VIEW [WORK].[vw_ProlongationAnalysis_OpenAndClosed_BalanceRub_WithRates]
AS
/*
  Это представление агрегирует данные по депозитам, начиная с 2024 года.
  
  Для открытых депозитов (определяемых по DT_OPEN) и закрытых (по DT_CLOSE_FACT):
  1. Отбираются депозиты с необходимыми фильтрами (CLI_SUBTYPE, PROD_NAME, DT_CLOSE_PLAN и
     прожили ≥ 10 дней).
  2. Для каждого депозита присоединяется информация из LIQUIDITY.[liq].man_CONVENTION (связь по 
     CONVENTION = CONVENTION_NAME) и рассчитывается ставка в ежемесячной конвенции посредством функции
     LIQUIDITY.liq.fnc_IntRate( RATE, NEW_CONVENTION_NAME, 'monthly', MATUR, 1 ).
     Эта ставка обозначается как ConvertedRate.
  3. Для открытых депозитов дополнительно определяется бакет срочности (на основе DaysLived) и 
     число пролонгаций (ProlongCount) через до 3 LEFT JOIN с [conrel_prolongations] (при условии, что
     предыдущие депозиты прожили ≥ 10 дней).
  4. Затем отдельно агрегируются показатели по открытым и по закрытым депозитам с сегментацией по:
         - Месяцу (MonthEnd),
         - Сегменту (с преобразованием: 'Розничный бизнес' → 'Розница', 'ДЧБО' → 'ЧБО', иначе 'Без сегментации'),
         - Валюте (CUR),
         - Бакету срочности (TERM_GROUP из man_TermGroup).
  5. Для открытых депозитов рассчитываются:
         - Количество и сумма BALANCE_RUB,
         - Количество и суммы по пролонгациям (общие, 1-я, 2-я, 3+),
         - Средневзвешенная ставка для всех открытых (WeightedRate_All) = Sum(ConvertedRate*BALANCE_RUB)/Sum(BALANCE_RUB),
         - А также отдельно для 1-й, 2-й и 3+ пролонгаций (WeightedRate_1y, WeightedRate_2y, WeightedRate_3plus),
           где в каждом случае вес берётся только по депозитам соответствующего ProlongCount.
  6. Для закрытых депозитов агрегируются количество и сумма BALANCE_RUB, а также рассчитывается 
     средневзвешенная ставка WeightedRate_Closed по аналогичной формуле.
  7. Итоговые агрегаты по открытым и закрытым депозитам объединяются через FULL OUTER JOIN по ключу 
     (MonthEnd, SegmentGrouping, CurrencyGrouping, TermBucketGrouping) – что позволяет получить итоговые
     строки, например «Все сегменты, все валюты, бакет '3 месяца'» и т.п.
  
  Все вычисления оборачиваются в ISNULL/COALESCE для защиты от NULL.
*/
WITH
-----------------------------
-- 0) Генерация месяцев (MonthEnd)
-----------------------------
cteMonths AS (
    SELECT CONVERT(date, '2024-01-31') AS MonthEnd
    UNION ALL
    SELECT EOMONTH(DATEADD(MONTH, 1, MonthEnd))
    FROM cteMonths
    WHERE EOMONTH(DATEADD(MONTH, 1, MonthEnd)) <= '2025-02-28'
),
-----------------------------
-- 1) Открытые депозиты с расчетом ConvertedRate
-----------------------------
DealsInMonthRates AS (
    SELECT
         M.MonthEnd,
         CAST(dc.CON_ID AS BIGINT) AS CON_ID,
         dc.SEG_NAME,
         dc.CUR,
         dc.DT_OPEN,
         dc.BALANCE_RUB,
         dc.RATE,
         dc.MATUR,
         conv.NEW_CONVENTION_NAME,
         DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) AS DaysLived,
         LIQUIDITY.liq.fnc_IntRate(dc.RATE, conv.NEW_CONVENTION_NAME, 'monthly', dc.MATUR, 1) AS ConvertedRate
    FROM cteMonths M
    JOIN [LIQUIDITY].[liq].[DepositContract_all] dc
         ON dc.DT_OPEN >= DATEADD(DAY, 1, EOMONTH(M.MonthEnd, -1))
        AND dc.DT_OPEN < DATEADD(DAY, 1, M.MonthEnd)
        AND dc.CLI_SUBTYPE = 'INDIV'
        AND dc.PROD_NAME   != 'Эскроу'
        AND dc.DT_CLOSE_PLAN != '4444-01-01'
        AND DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) >= 10
    JOIN LIQUIDITY.[liq].man_CONVENTION conv
         ON dc.CONVENTION = conv.CONVENTION_NAME
),
-----------------------------
-- 2) Присоединяем бакеты срочности для открытых депозитов
-----------------------------
DealsInMonthBucket AS (
    SELECT
         dmr.MonthEnd,
         dmr.CON_ID,
         dmr.SEG_NAME,
         dmr.CUR,
         dmr.DT_OPEN,
         dmr.BALANCE_RUB,
         dmr.DaysLived,
         dmr.ConvertedRate,
         tg.TERM_GROUP AS TermBucket
    FROM DealsInMonthRates dmr
    LEFT JOIN [ALM_TEST].[WORK].[man_TermGroup] tg
         ON dmr.DaysLived >= tg.TERM_FROM
        AND dmr.DaysLived <= tg.TERM_TO
),
-----------------------------
-- 3) Определяем число пролонгаций для открытых депозитов
-----------------------------
DealsWithProlong AS (
    SELECT
         dmb.MonthEnd,
         dmb.CON_ID,
         dmb.SEG_NAME,
         dmb.CUR,
         dmb.DT_OPEN,
         dmb.BALANCE_RUB,
         dmb.TermBucket,
         dmb.ConvertedRate,
         CASE
           WHEN p1.CON_ID IS NULL OR old1.CON_ID IS NULL THEN 0
           WHEN p2.CON_ID IS NULL OR old2.CON_ID IS NULL THEN 1
           WHEN p3.CON_ID IS NULL OR old3.CON_ID IS NULL THEN 2
           ELSE 3
         END AS ProlongCount
    FROM DealsInMonthBucket dmb
    LEFT JOIN [ALM].[ehd].[conrel_prolongations] p1
         ON p1.CON_ID = dmb.CON_ID
        AND p1.CON_REL_TYPE = 'PREVIOUS'
    LEFT JOIN [LIQUIDITY].[liq].[DepositContract_all] old1
         ON old1.CON_ID = p1.CON_ID_REL
        AND DATEDIFF(DAY, old1.DT_OPEN, old1.DT_CLOSE) >= 10
    LEFT JOIN [ALM].[ehd].[conrel_prolongations] p2
         ON p2.CON_ID = old1.CON_ID
        AND p2.CON_REL_TYPE = 'PREVIOUS'
    LEFT JOIN [LIQUIDITY].[liq].[DepositContract_all] old2
         ON old2.CON_ID = p2.CON_ID_REL
        AND DATEDIFF(DAY, old2.DT_OPEN, old2.DT_CLOSE) >= 10
    LEFT JOIN [ALM].[ehd].[conrel_prolongations] p3
         ON p3.CON_ID = old2.CON_ID
        AND p3.CON_REL_TYPE = 'PREVIOUS'
    LEFT JOIN [LIQUIDITY].[liq].[DepositContract_all] old3
         ON old3.CON_ID = p3.CON_ID_REL
        AND DATEDIFF(DAY, old3.DT_OPEN, old3.DT_CLOSE) >= 10
),
-----------------------------
-- 4) Агрегируем показатели по открытым депозитам
-----------------------------
OpenAggregated AS (
    SELECT 
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END AS SegmentGrouping,
         CUR AS CurrencyGrouping,
         TermBucket AS TermBucketGrouping,
         COUNT(*) AS OpenedDeals,
         SUM(BALANCE_RUB) AS Summ_BalanceRub,
         SUM(CASE WHEN ProlongCount > 0 THEN 1 ELSE 0 END) AS Count_Prolong,
         SUM(CASE WHEN ProlongCount = 1 THEN 1 ELSE 0 END) AS Count_1yProlong,
         SUM(CASE WHEN ProlongCount = 2 THEN 1 ELSE 0 END) AS Count_2yProlong,
         SUM(CASE WHEN ProlongCount >= 3 THEN 1 ELSE 0 END) AS Count_3plusProlong,
         SUM(CASE WHEN ProlongCount > 0 THEN BALANCE_RUB ELSE 0 END) AS Sum_ProlongRub,
         SUM(CASE WHEN ProlongCount = 1 THEN BALANCE_RUB ELSE 0 END) AS Sum_1yProlong_Rub,
         SUM(CASE WHEN ProlongCount = 2 THEN BALANCE_RUB ELSE 0 END) AS Sum_2yProlong_Rub,
         SUM(CASE WHEN ProlongCount >= 3 THEN BALANCE_RUB ELSE 0 END) AS Sum_3plusProlong_Rub,
         SUM(BALANCE_RUB * ConvertedRate) AS Sum_RateWeighted  -- для всех открытых
    FROM DealsWithProlong
    GROUP BY CUBE (
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END,
         CUR,
         TermBucket
    )
),
-----------------------------
-- 5) Расчет средневзвешенных ставок для открытых депозитов
-----------------------------
-- WeightedRate_All = Sum_RateWeighted / Summ_BalanceRub
-- Для отдельных пролонгаций: вычисляем сумму (BALANCE_RUB*ConvertedRate) только по тем записям,
-- где ProlongCount = X, и делим на сумму BALANCE_RUB по тем же записям.
OpenWeightedRates AS (
    SELECT 
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END AS SegmentGrouping,
         CUR AS CurrencyGrouping,
         TermBucket AS TermBucketGrouping,
         CASE WHEN SUM(BALANCE_RUB) > 0 THEN SUM(BALANCE_RUB * ConvertedRate) / SUM(BALANCE_RUB)
              ELSE 0 END AS WeightedRate_All,
         CASE WHEN SUM(CASE WHEN ProlongCount = 1 THEN BALANCE_RUB ELSE 0 END) > 0 
              THEN SUM(CASE WHEN ProlongCount = 1 THEN BALANCE_RUB * ConvertedRate ELSE 0 END)
                   / SUM(CASE WHEN ProlongCount = 1 THEN BALANCE_RUB ELSE 0 END)
              ELSE 0 END AS WeightedRate_1y,
         CASE WHEN SUM(CASE WHEN ProlongCount = 2 THEN BALANCE_RUB ELSE 0 END) > 0 
              THEN SUM(CASE WHEN ProlongCount = 2 THEN BALANCE_RUB * ConvertedRate ELSE 0 END)
                   / SUM(CASE WHEN ProlongCount = 2 THEN BALANCE_RUB ELSE 0 END)
              ELSE 0 END AS WeightedRate_2y,
         CASE WHEN SUM(CASE WHEN ProlongCount >= 3 THEN BALANCE_RUB ELSE 0 END) > 0 
              THEN SUM(CASE WHEN ProlongCount >= 3 THEN BALANCE_RUB * ConvertedRate ELSE 0 END)
                   / SUM(CASE WHEN ProlongCount >= 3 THEN BALANCE_RUB ELSE 0 END)
              ELSE 0 END AS WeightedRate_3plus
    FROM DealsWithProlong
    GROUP BY CUBE (
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END,
         CUR,
         TermBucket
    )
),
-----------------------------
-- 6) Закрытые депозиты (выходы) – по DT_CLOSE_FACT, с расчетом ConvertedRate для закрытых
-----------------------------
ClosedDealsInMonthRates AS (
    SELECT
         M.MonthEnd,
         CAST(dc.CON_ID AS BIGINT) AS CON_ID,
         dc.SEG_NAME,
         dc.CUR,
         dc.DT_OPEN,
         dc.DT_CLOSE_FACT,
         dc.BALANCE_RUB,
         dc.RATE,
         dc.MATUR,
         conv.NEW_CONVENTION_NAME,
         DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) AS DaysLived,
         LIQUIDITY.liq.fnc_IntRate(dc.RATE, conv.NEW_CONVENTION_NAME, 'monthly', dc.MATUR, 1) AS ConvertedRate
    FROM cteMonths M
    JOIN [LIQUIDITY].[liq].[DepositContract_all] dc
         ON dc.DT_CLOSE_FACT >= DATEADD(DAY, 1, EOMONTH(M.MonthEnd, -1))
         AND dc.DT_CLOSE_FACT < DATEADD(DAY, 1, M.MonthEnd)
         AND dc.CLI_SUBTYPE = 'INDIV'
         AND dc.PROD_NAME != 'ЭскроУ'
         AND dc.DT_CLOSE_PLAN != '4444-01-01'
         AND DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) >= 10
    JOIN LIQUIDITY.[liq].man_CONVENTION conv
         ON dc.CONVENTION = conv.CONVENTION_NAME
),
-----------------------------
-- 7) Присоединяем бакеты срочности для закрытых депозитов
-----------------------------
ClosedDealsInMonthBucket AS (
    SELECT
         cdm.MonthEnd,
         cdm.CON_ID,
         cdm.SEG_NAME,
         cdm.CUR,
         cdm.DT_OPEN,
         cdm.DT_CLOSE_FACT,
         cdm.BALANCE_RUB,
         cdm.DaysLived,
         cdm.ConvertedRate,
         tg.TERM_GROUP AS TermBucket
    FROM ClosedDealsInMonthRates cdm
    LEFT JOIN [ALM_TEST].[WORK].[man_TermGroup] tg
         ON cdm.DaysLived >= tg.TERM_FROM
         AND cdm.DaysLived <= tg.TERM_TO
),
-----------------------------
-- 8) Агрегируем показатели по закрытым депозитам
-----------------------------
ClosedAggregated AS (
    SELECT
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END AS SegmentGrouping,
         CUR AS CurrencyGrouping,
         TermBucket AS TermBucketGrouping,
         COUNT(*) AS ClosedDeals,
         SUM(BALANCE_RUB) AS Summ_ClosedBalanceRub,
         SUM(BALANCE_RUB * ConvertedRate) AS Sum_RateWeighted_Closed
    FROM ClosedDealsInMonthBucket
    GROUP BY CUBE (
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END,
         CUR,
         TermBucket
    )
),
-----------------------------
-- 9) Расчет средневзвешенной ставки для закрытых депозитов
-----------------------------
ClosedWeightedRates AS (
    SELECT
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END AS SegmentGrouping,
         CUR AS CurrencyGrouping,
         TermBucket AS TermBucketGrouping,
         CASE WHEN SUM(BALANCE_RUB) > 0 THEN SUM(BALANCE_RUB * ConvertedRate) / SUM(BALANCE_RUB)
              ELSE 0 END AS WeightedRate_Closed
    FROM ClosedDealsInMonthBucket
    GROUP BY CUBE (
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END,
         CUR,
         TermBucket
    )
)
-----------------------------
-- 10) Объединяем агрегаты по открытым и закрытым депозитам
-----------------------------
SELECT
    COALESCE(o.MonthEnd, c.MonthEnd) AS MonthEnd,
    ISNULL(COALESCE(o.SegmentGrouping, c.SegmentGrouping), N'Все сегменты') AS SegmentGrouping,
    ISNULL(COALESCE(o.CurrencyGrouping, c.CurrencyGrouping), N'Все валюты') AS CurrencyGrouping,
    ISNULL(COALESCE(o.TermBucketGrouping, c.TermBucketGrouping), N'Все бакеты') AS TermBucketGrouping,
    ISNULL(o.OpenedDeals, 0) AS OpenedDeals,
    ISNULL(o.Summ_BalanceRub, 0) AS Summ_BalanceRub,
    ISNULL(o.Count_Prolong, 0) AS Count_Prolong,
    ISNULL(o.Count_1yProlong, 0) AS Count_1yProlong,
    ISNULL(o.Count_2yProlong, 0) AS Count_2yProlong,
    ISNULL(o.Count_3plusProlong, 0) AS Count_3plusProlong,
    ISNULL(o.Sum_ProlongRub, 0) AS Sum_ProlongRub,
    ISNULL(o.Sum_1yProlong_Rub, 0) AS Sum_1yProlong_Rub,
    ISNULL(o.Sum_2yProlong_Rub, 0) AS Sum_2yProlong_Rub,
    ISNULL(o.Sum_3plusProlong_Rub, 0) AS Sum_3plusProlong_Rub,
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN
         1.0 * o.Sum_ProlongRub / o.Summ_BalanceRub
         ELSE 0 END AS [Доля_пролонгаций_Rub],
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN
         1.0 * o.Sum_1yProlong_Rub / o.Summ_BalanceRub
         ELSE 0 END AS [Доля_1йПролонгации_Rub],
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN
         1.0 * o.Sum_2yProlong_Rub / o.Summ_BalanceRub
         ELSE 0 END AS [Доля_2йПролонгации_Rub],
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN
         1.0 * o.Sum_3plusProlong_Rub / o.Summ_BalanceRub
         ELSE 0 END AS [Доля_3plusПролонгаций_Rub],
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN
         1.0 * o.Sum_RateWeighted / o.Summ_BalanceRub
         ELSE 0 END AS [WeightedRate_All],
    -- Для закрытых депозитов
    ISNULL(c.ClosedDeals, 0) AS ClosedDeals,
    ISNULL(c.Summ_ClosedBalanceRub, 0) AS Summ_ClosedBalanceRub,
    CASE WHEN ISNULL(c.Summ_ClosedBalanceRub,0) > 0 THEN
         1.0 * c.Sum_RateWeighted_Closed / c.Summ_ClosedBalanceRub
         ELSE 0 END AS [WeightedRate_Closed],
    -- Общее соотношение "выходов" относительно открытых
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN
         1.0 * c.Summ_ClosedBalanceRub / o.Summ_BalanceRub
         ELSE 0 END AS [Доля_выходов_Rub]
FROM OpenAggregated o
FULL OUTER JOIN ClosedAggregated c
    ON o.MonthEnd = c.MonthEnd
   AND o.SegmentGrouping = c.SegmentGrouping
   AND o.CurrencyGrouping = c.CurrencyGrouping
   AND o.TermBucketGrouping = c.TermBucketGrouping
ORDER BY
    COALESCE(o.MonthEnd, c.MonthEnd);
GO


SyntaxError: invalid character '≥' (U+2265) (3447331179.py, line 11)

In [ ]:
Ниже приведён пример расширённого представления, которое помимо ранее агрегированных показателей (количеств, сумм, долей пролонгаций и выходов) рассчитывает ещё средневзвешенные (в пересчёте в ежемесячной конвенции) ставки. При этом для каждого депозита из базы [liq].[DepositContract_all] выполняется преобразование его ставки через функцию  
LIQUIDITY.liq.fnc_IntRate( RATE, NEW_CONVENTION_NAME, 'monthly', MATUR, 1)  
(при этом таблица LIQUIDITY.[liq].man_CONVENTION связывается по полю CONVENTION_NAME, а из неё берётся поле NEW_CONVENTION_NAME). 

Средневзвешенная ставка (с учётом BALANCE_RUB) рассчитывается как сумма(ConvertedRate * BALANCE_RUB) / сумма(BALANCE_RUB). Кроме того, рассчитываются отдельно средневзвешенные ставки для всех открытых сделок, для тех, у которых имеется пролонгация (а также для 1-й, 2-й и 3+ пролонгаций) и для закрытых (выходов).

Ниже – подробный код представления. (Обратите внимание, что для упрощения я добавил два новых CTE – один для открытых сделок с расчётом ConvertedRate и один для закрытых, аналогично.)

```sql
USE ALM_TEST;
GO

CREATE OR ALTER VIEW [WORK].[vw_ProlongationAnalysis_OpenAndClosed_BalanceRub_WithRates]
AS
/*
  Это представление агрегирует данные по депозитам, начиная с 2024 года.
  
  Для открытых депозитов (определяемых по DT_OPEN) и закрытых (по DT_CLOSE_FACT):
  1. Отбираются депозиты с необходимыми фильтрами (CLI_SUBTYPE, PROD_NAME, DT_CLOSE_PLAN и
     прожили ≥ 10 дней).
  2. Для каждого депозита присоединяется информация из LIQUIDITY.[liq].man_CONVENTION (связь по 
     CONVENTION = CONVENTION_NAME) и рассчитывается ставка в ежемесячной конвенции посредством функции
     LIQUIDITY.liq.fnc_IntRate( RATE, NEW_CONVENTION_NAME, 'monthly', MATUR, 1 ).
     Эта ставка обозначается как ConvertedRate.
  3. Для открытых депозитов дополнительно определяется бакет срочности (на основе DaysLived) и 
     число пролонгаций (ProlongCount) через до 3 LEFT JOIN с [conrel_prolongations] (при условии, что
     предыдущие депозиты прожили ≥ 10 дней).
  4. Затем отдельно агрегируются показатели по открытым и по закрытым депозитам с сегментацией по:
         - Месяцу (MonthEnd),
         - Сегменту (с преобразованием: 'Розничный бизнес' → 'Розница', 'ДЧБО' → 'ЧБО', иначе 'Без сегментации'),
         - Валюте (CUR),
         - Бакету срочности (TERM_GROUP из man_TermGroup).
  5. Для открытых депозитов рассчитываются:
         - Количество и сумма BALANCE_RUB,
         - Количество и суммы по пролонгациям (общие, 1-я, 2-я, 3+),
         - Средневзвешенная ставка для всех открытых (WeightedRate_All) = Sum(ConvertedRate*BALANCE_RUB)/Sum(BALANCE_RUB),
         - А также отдельно для 1-й, 2-й и 3+ пролонгаций (WeightedRate_1y, WeightedRate_2y, WeightedRate_3plus),
           где в каждом случае вес берётся только по депозитам соответствующего ProlongCount.
  6. Для закрытых депозитов агрегируются количество и сумма BALANCE_RUB, а также рассчитывается 
     средневзвешенная ставка WeightedRate_Closed по аналогичной формуле.
  7. Итоговые агрегаты по открытым и закрытым депозитам объединяются через FULL OUTER JOIN по ключу 
     (MonthEnd, SegmentGrouping, CurrencyGrouping, TermBucketGrouping) – что позволяет получить итоговые
     строки, например «Все сегменты, все валюты, бакет '3 месяца'» и т.п.
  
  Все вычисления оборачиваются в ISNULL/COALESCE для защиты от NULL.
*/
WITH
-----------------------------
-- 0) Генерация месяцев (MonthEnd)
-----------------------------
cteMonths AS (
    SELECT CONVERT(date, '2024-01-31') AS MonthEnd
    UNION ALL
    SELECT EOMONTH(DATEADD(MONTH, 1, MonthEnd))
    FROM cteMonths
    WHERE EOMONTH(DATEADD(MONTH, 1, MonthEnd)) <= '2025-02-28'
),
-----------------------------
-- 1) Открытые депозиты с расчетом ConvertedRate
-----------------------------
DealsInMonthRates AS (
    SELECT
         M.MonthEnd,
         CAST(dc.CON_ID AS BIGINT) AS CON_ID,
         dc.SEG_NAME,
         dc.CUR,
         dc.DT_OPEN,
         dc.BALANCE_RUB,
         dc.RATE,
         dc.MATUR,
         conv.NEW_CONVENTION_NAME,
         DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) AS DaysLived,
         LIQUIDITY.liq.fnc_IntRate(dc.RATE, conv.NEW_CONVENTION_NAME, 'monthly', dc.MATUR, 1) AS ConvertedRate
    FROM cteMonths M
    JOIN [LIQUIDITY].[liq].[DepositContract_all] dc
         ON dc.DT_OPEN >= DATEADD(DAY, 1, EOMONTH(M.MonthEnd, -1))
        AND dc.DT_OPEN < DATEADD(DAY, 1, M.MonthEnd)
        AND dc.CLI_SUBTYPE = 'INDIV'
        AND dc.PROD_NAME   != 'Эскроу'
        AND dc.DT_CLOSE_PLAN != '4444-01-01'
        AND DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) >= 10
    JOIN LIQUIDITY.[liq].man_CONVENTION conv
         ON dc.CONVENTION = conv.CONVENTION_NAME
),
-----------------------------
-- 2) Присоединяем бакеты срочности для открытых депозитов
-----------------------------
DealsInMonthBucket AS (
    SELECT
         dmr.MonthEnd,
         dmr.CON_ID,
         dmr.SEG_NAME,
         dmr.CUR,
         dmr.DT_OPEN,
         dmr.BALANCE_RUB,
         dmr.DaysLived,
         dmr.ConvertedRate,
         tg.TERM_GROUP AS TermBucket
    FROM DealsInMonthRates dmr
    LEFT JOIN [ALM_TEST].[WORK].[man_TermGroup] tg
         ON dmr.DaysLived >= tg.TERM_FROM
        AND dmr.DaysLived <= tg.TERM_TO
),
-----------------------------
-- 3) Определяем число пролонгаций для открытых депозитов
-----------------------------
DealsWithProlong AS (
    SELECT
         dmb.MonthEnd,
         dmb.CON_ID,
         dmb.SEG_NAME,
         dmb.CUR,
         dmb.DT_OPEN,
         dmb.BALANCE_RUB,
         dmb.TermBucket,
         dmb.ConvertedRate,
         CASE
           WHEN p1.CON_ID IS NULL OR old1.CON_ID IS NULL THEN 0
           WHEN p2.CON_ID IS NULL OR old2.CON_ID IS NULL THEN 1
           WHEN p3.CON_ID IS NULL OR old3.CON_ID IS NULL THEN 2
           ELSE 3
         END AS ProlongCount
    FROM DealsInMonthBucket dmb
    LEFT JOIN [ALM].[ehd].[conrel_prolongations] p1
         ON p1.CON_ID = dmb.CON_ID
        AND p1.CON_REL_TYPE = 'PREVIOUS'
    LEFT JOIN [LIQUIDITY].[liq].[DepositContract_all] old1
         ON old1.CON_ID = p1.CON_ID_REL
        AND DATEDIFF(DAY, old1.DT_OPEN, old1.DT_CLOSE) >= 10
    LEFT JOIN [ALM].[ehd].[conrel_prolongations] p2
         ON p2.CON_ID = old1.CON_ID
        AND p2.CON_REL_TYPE = 'PREVIOUS'
    LEFT JOIN [LIQUIDITY].[liq].[DepositContract_all] old2
         ON old2.CON_ID = p2.CON_ID_REL
        AND DATEDIFF(DAY, old2.DT_OPEN, old2.DT_CLOSE) >= 10
    LEFT JOIN [ALM].[ehd].[conrel_prolongations] p3
         ON p3.CON_ID = old2.CON_ID
        AND p3.CON_REL_TYPE = 'PREVIOUS'
    LEFT JOIN [LIQUIDITY].[liq].[DepositContract_all] old3
         ON old3.CON_ID = p3.CON_ID_REL
        AND DATEDIFF(DAY, old3.DT_OPEN, old3.DT_CLOSE) >= 10
),
-----------------------------
-- 4) Агрегируем показатели по открытым депозитам
-----------------------------
OpenAggregated AS (
    SELECT 
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END AS SegmentGrouping,
         CUR AS CurrencyGrouping,
         TermBucket AS TermBucketGrouping,
         COUNT(*) AS OpenedDeals,
         SUM(BALANCE_RUB) AS Summ_BalanceRub,
         SUM(CASE WHEN ProlongCount > 0 THEN 1 ELSE 0 END) AS Count_Prolong,
         SUM(CASE WHEN ProlongCount = 1 THEN 1 ELSE 0 END) AS Count_1yProlong,
         SUM(CASE WHEN ProlongCount = 2 THEN 1 ELSE 0 END) AS Count_2yProlong,
         SUM(CASE WHEN ProlongCount >= 3 THEN 1 ELSE 0 END) AS Count_3plusProlong,
         SUM(CASE WHEN ProlongCount > 0 THEN BALANCE_RUB ELSE 0 END) AS Sum_ProlongRub,
         SUM(CASE WHEN ProlongCount = 1 THEN BALANCE_RUB ELSE 0 END) AS Sum_1yProlong_Rub,
         SUM(CASE WHEN ProlongCount = 2 THEN BALANCE_RUB ELSE 0 END) AS Sum_2yProlong_Rub,
         SUM(CASE WHEN ProlongCount >= 3 THEN BALANCE_RUB ELSE 0 END) AS Sum_3plusProlong_Rub,
         SUM(BALANCE_RUB * ConvertedRate) AS Sum_RateWeighted  -- для всех открытых
    FROM DealsWithProlong
    GROUP BY CUBE (
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END,
         CUR,
         TermBucket
    )
),
-----------------------------
-- 5) Расчет средневзвешенных ставок для открытых депозитов
-----------------------------
-- WeightedRate_All = Sum_RateWeighted / Summ_BalanceRub
-- Для отдельных пролонгаций: вычисляем сумму (BALANCE_RUB*ConvertedRate) только по тем записям,
-- где ProlongCount = X, и делим на сумму BALANCE_RUB по тем же записям.
OpenWeightedRates AS (
    SELECT 
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END AS SegmentGrouping,
         CUR AS CurrencyGrouping,
         TermBucket AS TermBucketGrouping,
         CASE WHEN SUM(BALANCE_RUB) > 0 THEN SUM(BALANCE_RUB * ConvertedRate) / SUM(BALANCE_RUB)
              ELSE 0 END AS WeightedRate_All,
         CASE WHEN SUM(CASE WHEN ProlongCount = 1 THEN BALANCE_RUB ELSE 0 END) > 0 
              THEN SUM(CASE WHEN ProlongCount = 1 THEN BALANCE_RUB * ConvertedRate ELSE 0 END)
                   / SUM(CASE WHEN ProlongCount = 1 THEN BALANCE_RUB ELSE 0 END)
              ELSE 0 END AS WeightedRate_1y,
         CASE WHEN SUM(CASE WHEN ProlongCount = 2 THEN BALANCE_RUB ELSE 0 END) > 0 
              THEN SUM(CASE WHEN ProlongCount = 2 THEN BALANCE_RUB * ConvertedRate ELSE 0 END)
                   / SUM(CASE WHEN ProlongCount = 2 THEN BALANCE_RUB ELSE 0 END)
              ELSE 0 END AS WeightedRate_2y,
         CASE WHEN SUM(CASE WHEN ProlongCount >= 3 THEN BALANCE_RUB ELSE 0 END) > 0 
              THEN SUM(CASE WHEN ProlongCount >= 3 THEN BALANCE_RUB * ConvertedRate ELSE 0 END)
                   / SUM(CASE WHEN ProlongCount >= 3 THEN BALANCE_RUB ELSE 0 END)
              ELSE 0 END AS WeightedRate_3plus
    FROM DealsWithProlong
    GROUP BY CUBE (
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END,
         CUR,
         TermBucket
    )
),
-----------------------------
-- 6) Закрытые депозиты (выходы) – по DT_CLOSE_FACT, с расчетом ConvertedRate для закрытых
-----------------------------
ClosedDealsInMonthRates AS (
    SELECT
         M.MonthEnd,
         CAST(dc.CON_ID AS BIGINT) AS CON_ID,
         dc.SEG_NAME,
         dc.CUR,
         dc.DT_OPEN,
         dc.DT_CLOSE_FACT,
         dc.BALANCE_RUB,
         dc.RATE,
         dc.MATUR,
         conv.NEW_CONVENTION_NAME,
         DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) AS DaysLived,
         LIQUIDITY.liq.fnc_IntRate(dc.RATE, conv.NEW_CONVENTION_NAME, 'monthly', dc.MATUR, 1) AS ConvertedRate
    FROM cteMonths M
    JOIN [LIQUIDITY].[liq].[DepositContract_all] dc
         ON dc.DT_CLOSE_FACT >= DATEADD(DAY, 1, EOMONTH(M.MonthEnd, -1))
         AND dc.DT_CLOSE_FACT < DATEADD(DAY, 1, M.MonthEnd)
         AND dc.CLI_SUBTYPE = 'INDIV'
         AND dc.PROD_NAME != 'ЭскроУ'
         AND dc.DT_CLOSE_PLAN != '4444-01-01'
         AND DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) >= 10
    JOIN LIQUIDITY.[liq].man_CONVENTION conv
         ON dc.CONVENTION = conv.CONVENTION_NAME
),
-----------------------------
-- 7) Присоединяем бакеты срочности для закрытых депозитов
-----------------------------
ClosedDealsInMonthBucket AS (
    SELECT
         cdm.MonthEnd,
         cdm.CON_ID,
         cdm.SEG_NAME,
         cdm.CUR,
         cdm.DT_OPEN,
         cdm.DT_CLOSE_FACT,
         cdm.BALANCE_RUB,
         cdm.DaysLived,
         cdm.ConvertedRate,
         tg.TERM_GROUP AS TermBucket
    FROM ClosedDealsInMonthRates cdm
    LEFT JOIN [ALM_TEST].[WORK].[man_TermGroup] tg
         ON cdm.DaysLived >= tg.TERM_FROM
         AND cdm.DaysLived <= tg.TERM_TO
),
-----------------------------
-- 8) Агрегируем показатели по закрытым депозитам
-----------------------------
ClosedAggregated AS (
    SELECT
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END AS SegmentGrouping,
         CUR AS CurrencyGrouping,
         TermBucket AS TermBucketGrouping,
         COUNT(*) AS ClosedDeals,
         SUM(BALANCE_RUB) AS Summ_ClosedBalanceRub,
         SUM(BALANCE_RUB * ConvertedRate) AS Sum_RateWeighted_Closed
    FROM ClosedDealsInMonthBucket
    GROUP BY CUBE (
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END,
         CUR,
         TermBucket
    )
),
-----------------------------
-- 9) Расчет средневзвешенной ставки для закрытых депозитов
-----------------------------
ClosedWeightedRates AS (
    SELECT
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END AS SegmentGrouping,
         CUR AS CurrencyGrouping,
         TermBucket AS TermBucketGrouping,
         CASE WHEN SUM(BALANCE_RUB) > 0 THEN SUM(BALANCE_RUB * ConvertedRate) / SUM(BALANCE_RUB)
              ELSE 0 END AS WeightedRate_Closed
    FROM ClosedDealsInMonthBucket
    GROUP BY CUBE (
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END,
         CUR,
         TermBucket
    )
)
-----------------------------
-- 10) Объединяем агрегаты по открытым и закрытым депозитам
-----------------------------
SELECT
    COALESCE(o.MonthEnd, c.MonthEnd) AS MonthEnd,
    ISNULL(COALESCE(o.SegmentGrouping, c.SegmentGrouping), N'Все сегменты') AS SegmentGrouping,
    ISNULL(COALESCE(o.CurrencyGrouping, c.CurrencyGrouping), N'Все валюты') AS CurrencyGrouping,
    ISNULL(COALESCE(o.TermBucketGrouping, c.TermBucketGrouping), N'Все бакеты') AS TermBucketGrouping,
    ISNULL(o.OpenedDeals, 0) AS OpenedDeals,
    ISNULL(o.Summ_BalanceRub, 0) AS Summ_BalanceRub,
    ISNULL(o.Count_Prolong, 0) AS Count_Prolong,
    ISNULL(o.Count_1yProlong, 0) AS Count_1yProlong,
    ISNULL(o.Count_2yProlong, 0) AS Count_2yProlong,
    ISNULL(o.Count_3plusProlong, 0) AS Count_3plusProlong,
    ISNULL(o.Sum_ProlongRub, 0) AS Sum_ProlongRub,
    ISNULL(o.Sum_1yProlong_Rub, 0) AS Sum_1yProlong_Rub,
    ISNULL(o.Sum_2yProlong_Rub, 0) AS Sum_2yProlong_Rub,
    ISNULL(o.Sum_3plusProlong_Rub, 0) AS Sum_3plusProlong_Rub,
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN
         1.0 * o.Sum_ProlongRub / o.Summ_BalanceRub
         ELSE 0 END AS [Доля_пролонгаций_Rub],
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN
         1.0 * o.Sum_1yProlong_Rub / o.Summ_BalanceRub
         ELSE 0 END AS [Доля_1йПролонгации_Rub],
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN
         1.0 * o.Sum_2yProlong_Rub / o.Summ_BalanceRub
         ELSE 0 END AS [Доля_2йПролонгации_Rub],
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN
         1.0 * o.Sum_3plusProlong_Rub / o.Summ_BalanceRub
         ELSE 0 END AS [Доля_3plusПролонгаций_Rub],
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN
         1.0 * o.Sum_RateWeighted / o.Summ_BalanceRub
         ELSE 0 END AS [WeightedRate_All],
    -- Для закрытых депозитов
    ISNULL(c.ClosedDeals, 0) AS ClosedDeals,
    ISNULL(c.Summ_ClosedBalanceRub, 0) AS Summ_ClosedBalanceRub,
    CASE WHEN ISNULL(c.Summ_ClosedBalanceRub,0) > 0 THEN
         1.0 * c.Sum_RateWeighted_Closed / c.Summ_ClosedBalanceRub
         ELSE 0 END AS [WeightedRate_Closed],
    -- Общее соотношение "выходов" относительно открытых
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN
         1.0 * c.Summ_ClosedBalanceRub / o.Summ_BalanceRub
         ELSE 0 END AS [Доля_выходов_Rub]
FROM OpenAggregated o
FULL OUTER JOIN ClosedAggregated c
    ON o.MonthEnd = c.MonthEnd
   AND o.SegmentGrouping = c.SegmentGrouping
   AND o.CurrencyGrouping = c.CurrencyGrouping
   AND o.TermBucketGrouping = c.TermBucketGrouping
ORDER BY
    COALESCE(o.MonthEnd, c.MonthEnd);
GO
```

### Ключевые моменты:
1. В CTE DealsInMonthRates для открытых депозитов происходит присоединение к LIQUIDITY.[liq].man_CONVENTION по полю CONVENTION, и рассчитывается ConvertedRate с помощью функции fnc_IntRate.
2. В CTE OpenAggregated агрегируются показатели по открытым депозитам (количество, сумма, количество пролонгаций, суммы по пролонгациям) и суммируется вес ставки (BALANCE_RUB * ConvertedRate) как Sum_RateWeighted.
3. Аналогично, в ClosedDealsInMonthRates для закрытых депозитов рассчитывается ConvertedRate.
4. В ClosedAggregated агрегируются показатели по закрытым депозитам, включая сумму взвешенных ставок Sum_RateWeighted_Closed.
5. Отдельный CTE OpenWeightedRates был объединён в агрегатные вычисления в OpenAggregated.
6. Итоговый SELECT объединяет агрегаты открытых и закрытых депозитов через FULL OUTER JOIN по четырём измерениям, и выводит итоговые показатели, включая средневзвешенную ставку для всех открытых (WeightedRate_All), для закрытых (WeightedRate_Closed) и соотношение выходов.
7. Использование GROUP BY CUBE в агрегатах гарантирует, что будут выведены итоговые строки типа «Все сегменты», «Все валюты», «Все бакеты» и т. д.

Этот код позволяет вам детально анализировать средневзвешенные ставки (в ежемесячной конвенции) по новым выдачам, пролонгациям (общая, 1-я, 2-я, 3+), а также по закрытым депозитам с сохранением требуемой сегментации.